In [2]:
# power = 260mW 
# T = 64 C 
# %matplotlib ipympl 
%matplotlib qt 
%load_ext autoreload 
%autoreload 2
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.dmd import DMD
import numpy as np 
from pianoq.misc.mplt import mimshow, mplot
import datetime 
import time 
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.lee_holo.generate_lee import main, make_oam_donut, orthogonal_lee

In [3]:
dmd = DMD()
dmd.set_white()

Connected!
Built project!


In [7]:
# grtn = 100
# dmd.set_grating(grtn)

# qq = 90
# dmd.set_checkerboard(qq)

# target = make_oam_donut(Nx=dmd.Nx, Ny=dmd.Ny, ell=2, r0=0.35, sigma=0.2)
# hologram, _ = orthogonal_lee(target, carrier_f=(1/50, 0), duty_cycle=0.5)
# dmd.set_image(hologram.astype(bool))

# amps = np.array([0, 0.5, 1])
# dmd.set_1d_macrosteps(amps, step_w=150, tile=8)

# dmd.set_black_between(450, 480)

mimshow(dmd.cur_image)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [20]:
# mimshow(dmd.cur_image)
print(dmd.last_called_setter)

In [22]:
mimshow(dmd.cur_image)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [13]:
timeout = 100000
x_motor = ThorlabsKcubeDC(serial_number=27253522, timeout=timeout)
y_motor = ThorlabsKcubeStepper(serial_number=26001271, timeout=timeout)

In [15]:
print(x_motor.get_position())
print(y_motor.get_position())

13.0
10.5


In [ ]:
integration_time = 4
# coin_window = 4e-9 
# ph = PhotonCounter(serial_port='COM8', integration_time=integration_time)
coin_window = 1.2e-9 # s 
# When DMD-plane fiber is 100um 1m. Delays=(0, 500) when both are 50um 2m fibers
tt = ph = QPTimeTagger(integration_time=integration_time, coin_window=coin_window, single_channel_delays=(0, 5000), remote=True, address='132.64.81.123:41101')  

In [7]:
tt.show_gui()

In [ ]:
tt.set_integration_time(1)
tt.show_gui()

nan issue in timetagger
nan issue in timetagger
nan issue in timetagger
nan issue in timetagger
nan issue in timetagger
nan issue in timetagger
nan issue in timetagger
nan issue in timetagger


In [9]:
s1, s2, c = ph.read_interesting()
s1, s2, c = s1.item(), s2.item(), c.item()
acc = 2*s1*s2*coin_window
real_c = c - acc 
std = np.sqrt(c) / np.sqrt(ph.integration_time)
print(f'{c=:.1f}; {real_c=:.1f}+-{std:.1f}')
print(x_motor.get_position())
print(y_motor.get_position())

c=200.7; real_c=18.5+-4.5
10.49948
12.4999992


In [27]:
x_motor.move_absolute(13.34)

In [16]:
# time.sleep(20)
start_x = 11.5 
start_y = 10.2 
x_pixels = 30
y_pixels = 3
pixel_size_x = 0.1  # mm
pixel_size_y = 0.1  # mm
name = f'grating_100_timetagger_z_12_fiber_100um_fiber_50um_260mW_f_none_before_crystal_filter_20nm'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\Projects\DMD\Alon results\Quantum\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=True, coin_window=coin_window, saveto_path=path)

single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 411584. Singles2: 199964. Coincidence: 214.00. real_coin: 16.5.
dur: 5. pix: 0, 1. Singles1: 411474. Singles2: 201160. Coincidence: 221.75. real_coin: 23.1.
dur: 10. pix: 0, 2. Singles1: 411544. Singles2: 203666. Coincidence: 224.75. real_coin: 23.6.
dur: 15. pix: 0, 3. Singles1: 410855. Singles2: 204370. Coincidence: 223.25. real_coin: 21.7.
dur: 20. pix: 0, 4. Singles1: 412112. Singles2: 203791. Coincidence: 212.00. real_coin: 10.4.
dur: 25. pix: 0, 5. Singles1: 412018. Singles2: 205133. Coincidence: 230.50. real_coin: 27.7.
dur: 30. pix: 0, 6. Singles1: 411824. Singles2: 203535. Coincidence: 213.00. real_coin: 11.8.
dur: 34. pix: 0, 7. Singles1: 411708. Singles2: 202971. Coincidence: 207.25. real_coin: 6.7.
dur: 39. pix: 0, 8. Singles1: 412213. Singles2: 202080. Coincidence: 224.75. real_coin: 24.8.
dur: 44. pix: 0, 9. Singles1: 414930. Singles2: 200286. Coincidence: 211.00. real_coin: 11.5.
dur: 49. pix: 0, 1

In [38]:
# time.sleep(10)
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 918209. Singles2: 197563. Coincidence: 469.80. real_coin: 34.4.
dur: 6. pix: 0, 1. Singles1: 918693. Singles2: 199704. Coincidence: 476.80. real_coin: 36.5.
dur: 12. pix: 0, 2. Singles1: 918508. Singles2: 198327. Coincidence: 465.20. real_coin: 28.0.
dur: 18. pix: 0, 3. Singles1: 917479. Singles2: 197796. Coincidence: 468.80. real_coin: 33.3.
dur: 24. pix: 0, 4. Singles1: 918606. Singles2: 193972. Coincidence: 481.00. real_coin: 53.4.
dur: 30. pix: 0, 5. Singles1: 918204. Singles2: 192048. Coincidence: 475.80. real_coin: 52.6.
dur: 36. pix: 0, 6. Singles1: 917969. Singles2: 193226. Coincidence: 451.60. real_coin: 25.9.
dur: 41. pix: 0, 7. Singles1: 919084. Singles2: 193453. Coincidence: 487.40. real_coin: 60.7.
dur: 47. pix: 0, 8. Singles1: 918539. Singles2: 194495. Coincidence: 483.00. real_coin: 54.2.
dur: 53. pix: 0, 9. Singles1: 918577. Singles2: 193758. Coincidence: 464.80. real_coin: 37.6.
dur: 59. pix: 0, 

In [27]:
mplot(scanner.result.real_coins.sum(axis=0))

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [24]:
# x_motor.close()
# y_motor.close()
# ph.close()
# dmd.close()

In [ ]:
# dmd.set_grating(100)
dmd.set_white()

## ASI cam 

In [4]:
cam = ASICam()

In [5]:
path = r'G:\My Drive\Projects\DMD\Alon results\Quantum'
# cam.set_roi(4300, 2100, 500, 500)
cam.set_exposure(3)
cam.set_roi(None, None, None, None)
cam.set_binning(4)
for i in [10, 30, 50, 70, 90, 110, 130, 150, 170, 190]:
    dmd.set_grating(i)
    # cam.show_image(vmin=None, vmax=None)
    cam.save_image(rf'{path}\farfield_plane_yes_lens_75mm_f1_200mm_3nm_grating{i}.fits')

In [36]:
cam.close()

In [ ]:
# pump with f=75mm 
mask = np.ones(dmd.shape, dtype=np.uint8)
mask[:200, :] = 0 
mask[700:, :] = 0 
mask[:, :350] = 0 
mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
mask = np.ones(dmd.shape, dtype=np.uint8)
# mask[:200, :] = 0 
# mask[700:, :] = 0 
# mask[:, :350] = 0 
# mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
r = 450
X0 = 0
Y0 = 0
mask = np.zeros(dmd.shape, dtype=np.uint8)
mask[np.sqrt((dmd.XX-X0)**2+(dmd.YY-Y0)**2) < r] = 1 
dmd.set_image(mask)
# mimshow(mask)